# Sensors Analytics REST API

## Deploy the API
- Go to the *Machine* tab, then set *Incoming connections* to **ON**. The API will be accessible through the indicated tunnelling link.  
- Run the notebook.



## Import the required modules

In [1]:
import cherrypy
import json
import redis

## Connect to the Redis Database

In [2]:
REDIS_HOST = "redis-17061.c300.eu-central-1-1.ec2.redns.redis-cloud.com"
REDIS_PORT = 17061
REDIS_USERNAME = 'default'
REDIS_PASSWORD = '2llMg9E9AkqLcDdx6HDpLQ7nzkQKgtCC'


redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


## Instructions:
1) Create a class for each endpoint (status, sensors, sensor)
    - Status
    - Sensors
    - Sensor
2) For each endpoint, implement the required HTTP methods (GET, POST, PUT, DELETE)
    - Status: GET
    - Sensors: GET, POST
    - Sensor: GET, PUT, DELETE
3) Map each object to its target endpoint.
    - Status() -> "/status"
    - Sensors() -> "/sensors"
    - Sensor() -> "/sensor"

## Status Endpoint Class

In [3]:
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

## Sensors Endpoint Class

In [4]:
class Sensors(object):
    exposed = True

    def GET(self, *path, **query):
        # print(query)
        min_t_samples = int(query.get('min_t_samples', 0))
        min_h_samples = int(query.get('min_h_samples', 0))
        sensors = []
        keys = redis_client.keys('0x*:temperature')

        count = 0
        for key in keys:
            key = key.decode()
            mac_address = key.split(':')[0]

            t_info = redis_client.ts().info(f'{mac_address}:temperature')
            t_samples = t_info.total_samples
            t_retention = t_info.retention_msecs
            h_info = redis_client.ts().info(f'{mac_address}:humidity')
            h_samples = h_info.total_samples
            h_retention = h_info.retention_msecs
            
            if t_samples >= min_t_samples and h_samples >= min_h_samples:
                sensors.append(
                    {
                        "mac_address": mac_address,
                        "t_samples": t_samples,
                        "t_retention": t_retention,
                        "h_samples": h_samples,
                        "h_retention": h_retention,
                    }
                )
                count += 1

        response_dict = {
            "sensors": sensors,
            "count": count,
        }

        response = json.dumps(response_dict)

        return response

    def POST(self, *path, **query):
        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        mac_address = body_dict.get('mac_address', None)

        if mac_address is None:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request body.')

        try:
            redis_client.ts().create(f'{mac_address}:temperature', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        try:
            redis_client.ts().create(f'{mac_address}:humidity', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        return

## Sensor Endpoint Class

In [5]:
class Sensor(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        try:
            t_info = redis_client.ts().info(f'{mac_address}:temperature')
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')
        
        h_info = redis_client.ts().info(f'{mac_address}:humidity')

        response_dict = {
            "mac_address": mac_address,
            "t_samples": t_info.total_samples,
            "t_retention": t_info.retention_msecs,
            "h_samples": h_info.total_samples,
            "h_retention": h_info.retention_msecs,
        }

        response = json.dumps(response_dict)

        return response

    def PUT(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        try:
            t_info = redis_client.ts().info(f'{mac_address}:temperature')
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        t_retention = body_dict.get('t_retention', None)

        if t_retention is None:
            raise cherrypy.HTTPError(400, 'Missing temperature retention period in the request body.')

        h_retention = body_dict.get('h_retention', None)

        if h_retention is None:
            raise cherrypy.HTTPError(400, 'Missing humidity retention period in the request body.')

        redis_client.ts().alter(f'{mac_address}:temperature', retention_msecs=t_retention)
        redis_client.ts().alter(f'{mac_address}:humidity', retention_msecs=h_retention)

        return
    
    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')
        
        mac_address = path[0]
        found = 0
        found += redis_client.delete(f'{mac_address}:temperature')
        found += redis_client.delete(f'{mac_address}:humidity')

        if found == 0:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        return

In [6]:
from datetime import datetime
class HistoricalData(object):

    exposed = True

    def GET(self, *path, **query):

        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: Missing MAC address in the request parameters.')

        mac_address = path[0]
        print(mac_address)
        
        if 'start_date' not in query:
            raise cherrypy.HTTPError(400, 'Bad Request: Missing start date in the request parameters.')
        if 'end_date' not in query:
            raise cherrypy.HTTPError(400, 'Bad Request: Missing end date in the request parameters.')

        start_date = query.get('start_date')
        end_date=query.get('end_date')

        try:
            start_date = datetime.fromisoformat(start_date)
        except ValueError:
            raise cherrypy.HTTPError(400, 'Bad Request: Wrong format for start date in the request parameters. ')

        try:
            end_date = datetime.fromisoformat(end_date)
        except ValueError:
            raise cherrypy.HTTPError(400, 'Bad Request: Wrong format for end date in the request parameters. ')

        end_date = int(datetime.timestamp(end_date)*1000)
        start_date = int(datetime.timestamp(start_date)*1000)

        if end_date <= start_date:
            raise cherrypy.HTTPError(400, 'Bad Request: End date smaller or equal than start date')

        try:
            temp_data = redis_client.ts().range(f'{mac_address}:temperature', start_date, end_date)
            humi_data = redis_client.ts().range(f'{mac_address}:humidity', start_date, end_date)
        except redis.ResponseError as e:
            raise cherrypy.HTTPError(404, 'Not Found: MAC address not found in the database.' + str(e))

        timestamps = [data[0] for data in temp_data]
        temps = [data[1] for data in temp_data]
        hums = [data[1] for data in humi_data]

        return json.dumps({
            'mac_address' : mac_address,
            'timestamps' : timestamps,
            'temperature' : temps,
            'humidity' : hums
        })


## Setup cherrypy and Map objects to their target endpoints

In [ ]:
if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(Sensors(), '/sensors', conf)
    cherrypy.tree.mount(Sensor(), '/sensor', conf)
    cherrypy.tree.mount(HistoricalData(), '/data', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[09/Jan/2025:08:14:50] ENGINE Bus STARTING
[09/Jan/2025:08:14:50] ENGINE Started monitor thread 'Autoreloader'.
[09/Jan/2025:08:14:50] ENGINE Serving on http://0.0.0.0:8080
[09/Jan/2025:08:14:50] ENGINE Bus STARTED
172.3.50.42 - - [09/Jan/2025:08:15:02] "GET /status HTTP/1.1" 200 20 "" "python-requests/2.32.3"
172.3.28.49 - - [09/Jan/2025:08:15:02] "POST /sensors HTTP/1.1" 409 3458 "" "python-requests/2.32.3"
172.3.186.144 - - [09/Jan/2025:08:15:02] "GET /sensors HTTP/1.1" 200 144 "" "python-requests/2.32.3"
172.3.186.144 - - [09/Jan/2025:08:15:03] "GET /sensor/{mac_address} HTTP/1.1" 404 3539 "" "python-requests/2.32.3"
{mac_address}
172.3.186.144 - - [09/Jan/2025:08:15:03] "GET /data/{mac_address}?start_date=2020-01-01&end_date=2028-04-10 HTTP/1.1" 404 3700 "" "python-requests/2.32.3"
172.3.28.49 - - [09/Jan/2025:08:15:34] "GET /status HTTP/1.1" 200 20 "" "python-requests/2.32.3"
172.3.50.42 - - [09/Jan/2025:08:15:34] "POST /sensors HTTP/1.1" 409 3458 "" "python-requests/2.32.3"
172.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b6b1fe17-4020-4f88-867f-48004baa1058' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>